In [2]:
import sqlite3
import pandas as pd
import os


# Cargamos una base de datos SQLite (Tabla "loans")

En este bloque de código se realizó la carga de los datos del archivo CSV en la base de datos SQLite de manera eficiente, utilizando la técnica de lectura por fragmentos (chunks) para manejar los 2 millones de registros sin saturar la memoria. Cada fragmento de 50.000 filas se lee con pandas.read_csv y se convierte en una tabla SQL mediante to_sql, creando automáticamente la tabla "loans" en la base de datos si aún no existía y agregando los registros en cada iteración. De esta forma, se logró almacenar toda la información de manera estructurada en SQLite, conservando todas las columnas del dataset original y permitiendo consultas posteriores sin necesidad de cargar el archivo completo en memoria.

In [3]:
conn = sqlite3.connect(r"C:\Users\User\Documents\GITHUB\final_project_creditscoring\Data\credit_scoring.db")
cursor = conn.cursor()

In [5]:


chunksize = 50000  # Ajusta según memoria disponible

for chunk in pd.read_csv(r"C:\Users\User\Documents\GITHUB\final_project_creditscoring\Data\accepted_2007_to_2018Q4.csv", chunksize=chunksize, parse_dates=['issue_d']):
 
    # chunk = chunk[(chunk['issue_d'] >= '2010-01-01') & (chunk['issue_d'] < '2018-01-01')]
    
    # Guardar en SQLite, creando la tabla automáticamente con todas las columnas
    chunk.to_sql("loans", conn, if_exists='append', index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\User\\Documents\\GITHUB\\final_project_creditscoring\\Data\\accepted_2007_to_2018Q4.csv'

En el siguiente bloque de código tiene como objetivo crear un dataset de muestra balanceado por mes a partir de la base de datos completa loans. Primero, se cuentan los registros de cada mes usando SQLite para identificar los meses que tienen al menos un mínimo de datos (MIN_REGISTROS_POR_MES). Luego, se calcula cuántos registros tomar de cada mes para que el dataset final tenga un total aproximado de TOTAL_REGISTROS. Después, se realiza un muestreo aleatorio por mes directamente desde la base de datos, asegurando que cada mes seleccionado tenga la misma cantidad de registros y que se excluyan los meses con pocos datos. Finalmente, se concatenan todos los fragmentos en un único DataFrame df_final, se convierten las fechas a formato datetime y se verifica que la forma del dataset y la cantidad de meses únicos sean correctas. Esto permite trabajar con un dataset más manejable y representativo de todos los meses con datos suficientes, sin cargar los 2 millones de registros originales en memoria.

In [9]:

TOTAL_REGISTROS = 200_000
MIN_REGISTROS_POR_MES = 1000

# 1️⃣ Obtener número de registros por mes
monthly_counts = pd.read_sql("""
    SELECT
        strftime('%Y-%m', issue_d) AS year_month,
        COUNT(*) AS n_registros
    FROM loans
    GROUP BY year_month
""", conn)

# 2️⃣ Filtrar meses con suficientes datos
valid_months = monthly_counts[
    monthly_counts['n_registros'] >= MIN_REGISTROS_POR_MES
]['year_month'].tolist()

# 3️⃣ Calcular cuántos registros tomar por mes
rows_per_month = TOTAL_REGISTROS // len(valid_months)

print(f"Meses válidos: {len(valid_months)}")
print(f"Registros por mes: {rows_per_month}")

# 4️⃣ Muestreo balanceado por mes desde SQLite
df_list = []

for m in valid_months:
    query = f"""
    SELECT *
    FROM loans
    WHERE strftime('%Y-%m', issue_d) = '{m}'
    ORDER BY RANDOM()
    LIMIT {rows_per_month}
    """
    df_month = pd.read_sql_query(query, conn)
    df_list.append(df_month)

# 5️⃣ Dataset final
df_final = pd.concat(df_list, ignore_index=True)

# 6️⃣ Conversión de fecha
df_final['issue_d'] = pd.to_datetime(df_final['issue_d'])


# 7️⃣ Verificación
print("Shape final:", df_final.shape)
print("Meses únicos:", df_final['issue_d'].dt.to_period('M').nunique())


Meses válidos: 103
Registros por mes: 1941


C:\Users\User\AppData\Local\Temp\ipykernel_20364\2995076181.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat(df_list, ignore_index=True)


Shape final: (192309, 151)
Meses únicos: 103


Confirmamos si hay meses faltantes y si los datos son continuos

In [10]:
# Obtener meses únicos ordenados
meses = (
    df_final['issue_d']
    .dt.to_period('M')
    .sort_values()
    .unique()
)

# Convertir a índice temporal
meses = pd.PeriodIndex(meses, freq='M')

# Crear rango completo esperado
rango_completo = pd.period_range(
    start=meses.min(),
    end=meses.max(),
    freq='M'
)

# Detectar meses faltantes
meses_faltantes = rango_completo.difference(meses)

# Resultado
if len(meses_faltantes) == 0:
    print("✅ No hay meses faltantes. La serie es continua.")
else:
    print("⚠️ Hay meses sin datos:")
    print(meses_faltantes)


✅ No hay meses faltantes. La serie es continua.


# Guardar una tabla llamada 'main_table' en nuestro archivo credit_scoring.db
Por ultimo guardamos en una tabla el data set anteriormente creado.

In [4]:

df_final.to_sql("main_table", conn, if_exists="replace", index=False)

conn.close()

NameError: name 'df_final' is not defined